# **Preparations**

### **Importing necessary modules**

In [ ]:
import psycopg2
import os

### **Estabilishing connection and declaring the cursor**

In [ ]:
def connection():
    conn = psycopg2.connect(
        dbname= 'projeto1',
        user= 'postgres',
        password= '',   # Replace with actual password
        host= 'localhost',
        port= '5432'
    )
    cursor = conn.cursor()
    return conn, cursor

# **Creating all tables**

In [159]:
def create_tables_sql():
    create_tables_sql = """
        BEGIN;


        ALTER TABLE IF EXISTS public."BasicHealthUnit" DROP CONSTRAINT IF EXISTS None;

        ALTER TABLE IF EXISTS public."Death" DROP CONSTRAINT IF EXISTS None;

        ALTER TABLE IF EXISTS public."Death" DROP CONSTRAINT IF EXISTS None;

        ALTER TABLE IF EXISTS public."Death" DROP CONSTRAINT IF EXISTS None;

        ALTER TABLE IF EXISTS public."Mother" DROP CONSTRAINT IF EXISTS "Mother_mother_id_fkey";

        ALTER TABLE IF EXISTS public."Municipality" DROP CONSTRAINT IF EXISTS "Municipality_IBGE_state_code_fkey";



        DROP TABLE IF EXISTS public."BasicHealthUnit";

        CREATE TABLE IF NOT EXISTS public."BasicHealthUnit"
        (
            "CNES" integer NOT NULL,
            name character varying(200) COLLATE pg_catalog."default",
            "IBGE_city_code" integer,
            CONSTRAINT "BasicHealthUnit_pkey" PRIMARY KEY ("CNES")
        );

        DROP TABLE IF EXISTS public."Death";

        CREATE TABLE IF NOT EXISTS public."Death"
        (
            death_id integer NOT NULL,
            date date,
            "time" time without time zone,
            location_code integer,
            investigated boolean,
            basic_cause_id character varying(10) COLLATE pg_catalog."default",
            time_relative_birth integer,
            pregnancy_type integer,
            gestational_age integer,
            birth_type integer,
            CONSTRAINT "Death_pkey" PRIMARY KEY (death_id)
        );

        DROP TABLE IF EXISTS public."DeathCause";

        CREATE TABLE IF NOT EXISTS public."DeathCause"
        (
            basic_cause_id character varying(10) COLLATE pg_catalog."default" NOT NULL,
            description character varying(500) COLLATE pg_catalog."default",
            CONSTRAINT "DeathCause_pkey" PRIMARY KEY (basic_cause_id)
        );

        DROP TABLE IF EXISTS public."Deceased";

        CREATE TABLE IF NOT EXISTS public."Deceased"
        (
            deceased_id integer NOT NULL,
            occupation integer,
            marital_status integer,
            age double precision,
            weight double precision,
            education_level integer,
            race_color integer,
            sex integer,
            CONSTRAINT "Deceased_pkey" PRIMARY KEY (deceased_id)
        );

        DROP TABLE IF EXISTS public."Mother";

        CREATE TABLE IF NOT EXISTS public."Mother"
        (
            mother_id integer NOT NULL,
            living_children integer,
            deceased_children integer,
            education_level integer,
            age double precision,
            occupation integer,
            CONSTRAINT "Mother_pkey" PRIMARY KEY (mother_id)
        );

        DROP TABLE IF EXISTS public."Municipality";

        CREATE TABLE IF NOT EXISTS public."Municipality"
        (
            "IBGE_city_code" integer NOT NULL,
            name character varying(200) COLLATE pg_catalog."default" NOT NULL,
            "GDP" real,
            "GDP_per_capita" real,
            population_estimate_2009 double precision,
            "IBGE_state_code" integer,
            CONSTRAINT "Municipality_pkey" PRIMARY KEY ("IBGE_city_code")
        );

        DROP TABLE IF EXISTS public."State";

        CREATE TABLE IF NOT EXISTS public."State"
        (
            "IBGE_state_code" integer NOT NULL,
            name character varying(200) COLLATE pg_catalog."default",
            CONSTRAINT "State_pkey" PRIMARY KEY ("IBGE_state_code")
        );

        ALTER TABLE IF EXISTS public."BasicHealthUnit"
            ADD FOREIGN KEY ("IBGE_city_code")
            REFERENCES public."Municipality" ("IBGE_city_code") MATCH SIMPLE
            ON UPDATE NO ACTION
            ON DELETE NO ACTION
            NOT VALID;


        ALTER TABLE IF EXISTS public."Death"
            ADD FOREIGN KEY (basic_cause_id)
            REFERENCES public."DeathCause" (basic_cause_id) MATCH SIMPLE
            ON UPDATE NO ACTION
            ON DELETE NO ACTION
            NOT VALID;


        ALTER TABLE IF EXISTS public."Death"
            ADD FOREIGN KEY (death_id)
            REFERENCES public."Deceased" (deceased_id) MATCH SIMPLE
            ON UPDATE NO ACTION
            ON DELETE NO ACTION
            NOT VALID;


        ALTER TABLE IF EXISTS public."Death"
            ADD FOREIGN KEY (location_code)
            REFERENCES public."Municipality" ("IBGE_city_code") MATCH SIMPLE
            ON UPDATE NO ACTION
            ON DELETE NO ACTION
            NOT VALID;


        ALTER TABLE IF EXISTS public."Mother"
            ADD CONSTRAINT "Mother_mother_id_fkey" FOREIGN KEY (mother_id)
            REFERENCES public."Deceased" (deceased_id) MATCH SIMPLE
            ON UPDATE NO ACTION
            ON DELETE NO ACTION
            NOT VALID;
        CREATE INDEX IF NOT EXISTS "Mother_pkey"
            ON public."Mother"(mother_id);


        ALTER TABLE IF EXISTS public."Municipality"
            ADD CONSTRAINT "Municipality_IBGE_state_code_fkey" FOREIGN KEY ("IBGE_state_code")
            REFERENCES public."State" ("IBGE_state_code") MATCH SIMPLE
            ON UPDATE NO ACTION
            ON DELETE NO ACTION
            NOT VALID;

        END;
        """
    return create_tables_sql

# **Inserting data into the tables**

### **Inserting data into the "BasicHealthUnit" table**

In [160]:
def insert_ubs(cursor):
    insert_ubs_sql = """
        COPY "BasicHealthUnit" ("CNES", NAME, "IBGE_city_code")
        FROM STDIN WITH (
            FORMAT CSV,
            HEADER,
            DELIMITER ';',
            NULL ''
        )
    """
    with open(os.path.abspath('../preprocessed_datasets/BasicHealthUnits.csv'), 'r', encoding='utf-8') as file:
        cursor.copy_expert(insert_ubs_sql, file)

### **Inserting data into the "State" table**

In [ ]:
def insert_state(cursor):
    insert_state_sql = """
        COPY "State" ("IBGE_state_code", NAME)
        FROM STDIN WITH (
            FORMAT CSV,
            HEADER,
            DELIMITER ';',
            NULL ''
        )
    """
    with open(os.path.abspath('../preprocessed_datasets/State.csv'), 'r', encoding='utf-8') as file:
        cursor.copy_expert(insert_state_sql, file)

### **Inserting data into the "Municipality" table**

In [162]:
def insert_municipality(cursor):
    insert_munincipality_sql = """
        COPY "Municipality" ("IBGE_city_code", NAME, "GDP", "GDP_per_capita", POPULATION_ESTIMATE_2009, "IBGE_state_code")
        FROM STDIN WITH (
            FORMAT CSV,
            HEADER,
            DELIMITER ';',
            NULL ''
        )
    """
    with open(os.path.abspath('../preprocessed_datasets/Municipality.csv'), 'r', encoding='utf-8') as file:
        cursor.copy_expert(insert_munincipality_sql, file)

### **Inserting data into the "Death" table**

In [ ]:
def insert_death(cursor):
    insert_death_sql = """
        COPY "Death" (DEATH_ID, DATE, TIME, LOCATION_CODE, INVESTIGATED, BASIC_CAUSE_ID, TIME_RELATIVE_BIRTH, PREGNANCY_TYPE, GESTATIONAL_AGE, BIRTH_TYPE)
        FROM STDIN WITH (
            FORMAT CSV,
            HEADER,
            DELIMITER ';',
            NULL ''
        )
    """
    with open(os.path.abspath('../preprocessed_datasets/Death.csv'), 'r', encoding='utf-8') as file:
        cursor.copy_expert(insert_death_sql, file)

### **Inserting data into the "DeathCause" table**

In [164]:
def insert_death_cause(cursor):
    insert_death_cause_sql = """
        COPY "DeathCause" (BASIC_CAUSE_ID, DESCRIPTION)
        FROM STDIN WITH (
            FORMAT CSV,
            HEADER,
            DELIMITER ';',
            NULL ''
        )
    """
    with open(os.path.abspath('../preprocessed_datasets/DeathCause.csv'), 'r', encoding='ISO-8859-1') as file:
        cursor.copy_expert(insert_death_cause_sql, file)

### **Inserting data into the "Mother" table**

In [ ]:
def insert_mother(cursor):
    insert_mother_sql = """
        COPY "Mother" (MOTHER_ID, LIVING_CHILDREN, DECEASED_CHILDREN, EDUCATION_LEVEL, AGE, OCCUPATION)
        FROM STDIN WITH (
            FORMAT CSV,
            HEADER,
            DELIMITER ';',
            NULL ''
        )
    """
    with open(os.path.abspath('../preprocessed_datasets/Mother.csv'), 'r', encoding='utf-8') as file:
        cursor.copy_expert(insert_mother_sql, file)

### **Inserting data into the "Deceased" table**

In [ ]:
def insert_deceased(cursor):
    insert_deceased_sql = """
        COPY "Deceased" (DECEASED_ID, OCCUPATION, MARITAL_STATUS, AGE, WEIGHT, EDUCATION_LEVEL, RACE_COLOR, SEX)
        FROM STDIN WITH (
            FORMAT CSV,
            HEADER,
            DELIMITER ';',
            NULL ''
        )
    """
    with open(os.path.abspath('../preprocessed_datasets/Deceased.csv'), 'r', encoding='utf-8') as file:
        cursor.copy_expert(insert_deceased_sql, file)

# **Executing the scripts and commiting changes**

### **Executing SQL scripts**

In [167]:
def execute_script(cursor, script):
    cursor.execute(script)

### **Commiting changes and closing the cursor and connection**

In [168]:
def close_conn(connection, cursor):
    connection.commit()
    cursor.close()
    connection.close()

# **Main**

In [169]:
conn,cursor = connection()
sql = create_tables_sql()
cursor.execute(sql)
conn.commit()
insert_state(cursor)
insert_municipality(cursor)
insert_deceased(cursor)
insert_mother(cursor)
insert_death_cause(cursor)
insert_ubs(cursor)
insert_death(cursor)
close_conn(conn, cursor)